In [1]:
import filepath_

In [2]:
import torch
import numpy as np
import sympy as sp
from abc import ABC
from typing import List, Dict, Tuple
from panda3d.core import VBase3
from panda3d.core import Vec3
from panda3d.core import Quat
from panda3d.core import invert

from panda3d_game.game_object import GameObject, PhysicsGameObject
from panda3d.core import TransformState
# 
from util.geometry import batch_transform, createPosIndicatorNPth
from art.basic import geom_frm_faces, create_cylinder_node, create_cylinder
from panda3d.core import GeomPrimitive
from panda3d.core import (
    Geom,
    GeomNode,
    GeomTriangles,
    GeomVertexData,
    GeomVertexFormat,
    GeomVertexWriter,
    GeomEnums, Vec3, LPoint3f
)    
from panda3d.core import GeomNode, NodePath, GeomVertexReader, BitMask32
from art.basic import uv_curve_surface, uv_curve_surface_lambda
from panda3d.bullet import (
    BulletWorld, BulletRigidBodyNode, BulletTriangleMesh, 
    BulletTriangleMeshShape, BulletDebugNode,
      BulletHingeConstraint, BulletGenericConstraint,
    BulletShape, BulletMinkowskiSumShape,BulletCylinderShape,
BulletConvexHullShape,BulletTriangleMeshShape, BulletTriangleMesh
)
from panda3d_game.constraints import FixedConstraint
from panda3d_game.app import UniversalGravitySpace
from qpanda3d import QControl
from ui.qtui import *
from panda3d.bullet import BulletRigidBodyNode
from panda3d.bullet import BulletSphereShape, BulletCylinderShape
from console.physics_room_console import PhyscRoomConsole

In [3]:
from icosahedron import *
from panda3d_spherical import *
from panda3d.core import DirectionalLight, Vec3, PointLight
from sympy.physics.units import (
    kilometer, meter,centimeter,
    gram, kilogram, tonne,
    newton, second
)

In [4]:
# format = 

In [5]:
mesh = SphereMeshObject(R=100)

In [6]:
mesh.uniformSplit(5)

iter 0
iter 1
iter 2
iter 3
iter 4


In [7]:
mesh.toGeom()

Geom [ GeomTriangles ], 20480 faces

In [7]:
"""
:gobj(error): GeomTriangles references vertices up to 64982, but GeomVertexData has only 48738 rows!
Assertion failed: primitive->check_valid(cdata->_data.get_read_pointer(current_thread)) at line 364 of panda/src/gobj/geom.cxx
"""

'\n:gobj(error): GeomTriangles references vertices up to 64982, but GeomVertexData has only 48738 rows!\nAssertion failed: primitive->check_valid(cdata->_data.get_read_pointer(current_thread)) at line 364 of panda/src/gobj/geom.cxx\n'

In [8]:
len(mesh.verts)

64787

In [9]:
x = sp.Symbol('x')
e = (64984*x)/ (48738*x)
e.simplify()
# seems that it is format issue

4/3

In [ ]:
mesh.getXYZ().shape

In [9]:
tp = mesh.getThetaPhi()


In [19]:
xyz = sphr2cart_pt(tp[:,0], tp[:,1])
xyz * mesh.getHeight()

RuntimeError: The size of tensor a (3) must match the size of tensor b (64898) at non-singleton dimension 1

In [20]:
xyz.shape

torch.Size([64898, 3])

In [8]:

        

def addHyperEdgeAsTriangle(hp:HyperEdge):
    rho = 1
    x = hp.x
    p = x.theta
    y = hp.y
    z = hp.z
    

class ShipScene(
    # StarScene, 
    UniversalGravitySpace, 
    QControl
    # ControlShowBase
):
    def __init__(self, num_iterations=20):
        # StarScene.__init__(self)
        unit = {
            "mass" : tonne,
            "length" : 10*meter,
            "time": 1 * second,
            # "force" : sp.Number(1e3) * newton
        }
        G_game = 0.001
        UniversalGravitySpace.__init__(self, unit, G_game, 20, 1/(60*20))
        QControl.__init__(self)
       
       
        point_light = PointLight('light')
        
        point_light.setColor((1, 1, 1, 1))  # 设置光源颜色 (白色)
        point_light.setShadowCaster(True)
        light_np = self.render.attachNewNode(point_light)
        light_np.setPos(2, 2, 2)
        self.render.setLight(light_np)  # 将光源应用到场景
        
        # point_light.set_sc
        dire_light =DirectionalLight('light')
        dire_light.setDirection((-1,-1,-1))
        dire_light.set_color((1, 1, 1, 1.0)) 
        dire_light.setShadowCaster(True)
        light_dr = render.attachNewNode(dire_light)
        # light_dr.setPos(1, 2, 6)  # 设置光源位置
        
        self.render.setShaderAuto()
        self.render.setLight(light_dr)
        self.startQt()
        self.m = mesh
        self.m.rigid_np.reparentTo(self.rdr_scene)
        self.bullet_world.attachRigidBody(mesh.rigid_node)

        # self.ship.toBulletWorld(self.bullet_world)
        # self.shell.toBulletWorld(self.bullet_world)
        # self.planet1.toBulletWorld(self.bullet_world)
        # self.taskMgr.add(self.update_torque_)
       

        debug_node = BulletDebugNode('Debug')
        debug_node.showWireframe(True)
        debug_node.showConstraints(True)
        debug_node.showBoundingBoxes(False)
        debug_node.showNormals(False)
        debug_np = self.rdr_scene.attach_new_node(debug_node)
        self.bullet_world.set_debug_node(debug_node)
        debug_np.show()
class ShipView(RawQtGUI):
    def get_game(self):
        return ShipScene()

    def get_console(self):
        return PhyscRoomConsole(showbase=self.panda3d) 

In [ ]:
if __name__ == '__main__':
    # torch.set_printoptions(precision=16, sci_mode=False)
    import sys
    app = QApplication(sys.argv)
    window = ShipView()
    window.show()
    sys.exit(app.exec_())

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
